# Data Load

In [1]:
import config
from pshmodule.utils import filemanager as fm

In [2]:
df = fm.load(config.temp_data)

extension : .pickle
Loaded 9150 records from /Volumes/GoogleDrive/내 드라이브/MemeProject/data/temp_for_doc2vec.pickle


In [3]:
df.head(20)

1,content,pos
0,나 중간고사 반에서 1등했어,"[나, 중간고사, 반]"
1,중간고사 점수 내가 반에서 제일 잘 받음,"[중간고사, 점수, 내, 반, 제일, 받음]"
2,나 반에서 중간 성적 제일 좋아,"[나, 반, 중간, 성적, 제일, 좋아]"
3,우리 반에서 내가 시험 제일 잘 봤다,"[우리, 반, 내, 시험, 제일, 봤다]"
4,중간고사 반 1등 먹음,"[중간고사, 반, 먹음]"
5,중간시험 내가 반 1등이야,"[중간, 시험, 내, 반]"
6,너 왜 이렇게 멍청하냐?,"[너, 왜, 이렇게, 멍청하냐]"
7,이 새끼 빡대가리잖아?,"[새끼, 빡, 대가리, 잖아]"
8,너 말귀 못 알아듣냐고,"[너, 말귀, 못, 알아듣냐고]"
9,병신 이해력 딸리네,"[병신, 이해력, 딸리네]"


# 

# word2vec

In [4]:
from gensim.models import Word2Vec
from gensim.models.callbacks import CallbackAny2Vec

### loss logger

In [5]:
class LossLogger(CallbackAny2Vec):
    '''Output loss at each epoch'''
    def __init__(self):
        self.epoch = 1
        self.losses = []

    def on_epoch_begin(self, model):
        print(f'Epoch: {self.epoch}', end='\t')

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        # self.losses.append(loss)
        print(f'  Loss: {loss}')
        self.epoch += 1

### train

In [6]:
print('\tPreparing model with the following parameters: epochs = {}, vector_size = {}, alpha = {} .. '.
      format(config.max_epochs, config.vec_size, config.alpha))

model = Word2Vec(sentences=df.pos,
                 vector_size=config.vec_size,
                 window=config.window,
                 min_count=config.min_count,
                 workers=config.workers,
                 epochs=config.max_epochs,
                 sg = 1) # 0: CBOW, 1: skip-gram

model.build_vocab(df.pos)

	Preparing model with the following parameters: epochs = 100, vector_size = 512, alpha = 0.01 .. 


In [7]:
model.train(df.pos,
            epochs=model.epochs,
            total_examples=len(df.pos),
            compute_loss=True,
            callbacks=[LossLogger()]
           )

Epoch: 1	  Loss: 130311.2890625
Epoch: 2	  Loss: 240692.328125
Epoch: 3	  Loss: 342925.9375
Epoch: 4	  Loss: 437016.0
Epoch: 5	  Loss: 524745.9375
Epoch: 6	  Loss: 605579.75
Epoch: 7	  Loss: 680939.25
Epoch: 8	  Loss: 750571.875
Epoch: 9	  Loss: 815255.625
Epoch: 10	  Loss: 876532.0
Epoch: 11	  Loss: 932936.625
Epoch: 12	  Loss: 986104.0625
Epoch: 13	  Loss: 1036425.0625
Epoch: 14	  Loss: 1082790.625
Epoch: 15	  Loss: 1127503.75
Epoch: 16	  Loss: 1169505.75
Epoch: 17	  Loss: 1210038.25
Epoch: 18	  Loss: 1249308.875
Epoch: 19	  Loss: 1286791.125
Epoch: 20	  Loss: 1323080.0
Epoch: 21	  Loss: 1357423.625
Epoch: 22	  Loss: 1391893.25
Epoch: 23	  Loss: 1425412.625
Epoch: 24	  Loss: 1458295.75
Epoch: 25	  Loss: 1490273.625
Epoch: 26	  Loss: 1521293.875
Epoch: 27	  Loss: 1551996.0
Epoch: 28	  Loss: 1582449.375
Epoch: 29	  Loss: 1612056.5
Epoch: 30	  Loss: 1641388.125
Epoch: 31	  Loss: 1670292.0
Epoch: 32	  Loss: 1698342.875
Epoch: 33	  Loss: 1726422.125
Epoch: 34	  Loss: 1753724.25
Epoch: 35	

(4195698, 4601800)

# 

# model save

In [18]:
model.save(config.word2vec)
print("Model Saved")

Model Saved


# 